## FEATURE EXTRACTION FROM TEXT

In [3]:
# Perform imports and load the dataset:
import numpy as np
import pandas as pd

df = pd.read_csv('C:/Users/ksa/Downloads/original/UPDATED_NLP_COURSE/TextFiles/smsspamcollection.tsv', sep='\t')
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [5]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [7]:
df.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X = df['message']
y = df['label']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .30, random_state = 42)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
count_vect = CountVectorizer()

In [21]:
# FIT Vectorizer to data (build a vocab, count # of words....)
# Transform the original text message ---> Vector
X_train_counts = count_vect.fit_transform(X_train)

In [22]:
# So this shows that 3900 messages have 7263 unique words
X_train_counts

<3900x7263 sparse matrix of type '<class 'numpy.int64'>'
	with 52150 stored elements in Compressed Sparse Row format>

In [19]:
# Pass Countvectorizer to TFIDF transformer ( to give more weights to important words and penalize repitive words)
from sklearn.feature_extraction.text import TfidfTransformer

In [20]:
tfidf_transformer = TfidfTransformer()

In [23]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [25]:
X_train_tfidf.shape

(3900, 7263)

In [26]:
# Now a quicker and better way to do both countvectorizer and transformer - USE TFIDFVECTORIZER
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

In [28]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [30]:
X_train_tfidf.shape

(3900, 7263)

In [31]:
# Model Training for classification - Linear SVC
from sklearn.svm import LinearSVC

In [32]:
# Create classification model instance
clf = LinearSVC()

In [34]:
# fit the vectorized text data
clf.fit(X_train_tfidf, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

## Use pipeline feature to run everything in one code - vectorization and classification

In [35]:
from sklearn.pipeline import Pipeline

In [36]:
text_classifier = Pipeline([('tfidf', TfidfVectorizer()),('classifier', LinearSVC())])

In [37]:
text_classifier.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
        

In [38]:
predictions = text_classifier.predict(X_test)

In [40]:
from sklearn.metrics import confusion_matrix, classification_report

In [43]:
print(confusion_matrix(y_test, predictions))

[[1445    3]
 [  10  214]]


In [45]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      1.00      1448
        spam       0.99      0.96      0.97       224

    accuracy                           0.99      1672
   macro avg       0.99      0.98      0.98      1672
weighted avg       0.99      0.99      0.99      1672



In [51]:
text_classifier.predict(["Congratulations you have won a lottery of $10000"])

array(['spam'], dtype=object)